In [11]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [88]:
df_sample = pd.read_csv('/home/centos/leon/machine_learning_jupyter/dataset/jinnan_round1_train_20181227.csv')
df_sample = pd.DataFrame(df_sample.values.tolist(), columns=(['id'] + df_sample.iloc[:, 1:-1].columns.tolist() + ['y']))
print(df_sample.shape)
df_sample.head()

(1396, 44)


,id,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,B6,B7,B8,B9,B10,B11,B12,B13,B14,y
0,sample_1528,300,NaN,405.0,700,13:30:00,38.0,NaN,NaN,15:30:00,...,65,11:30:00,45.0,11:30-13:00,14:00-15:30,NaN,800.0,0.15,400,0.879
1,sample_1698,300,NaN,405.0,700,14:00:00,29.0,NaN,NaN,16:00:00,...,80,6:00:00,45.0,6:00-7:30,7:30-9:00,9:00-10:00,1200.0,0.15,400,0.902
2,sample_639,300,NaN,405.0,700,14:00:00,29.0,NaN,NaN,16:00:00,...,80,1:00:00,45.0,1:00-2:30,2:30-4:00,4:00-5:00,1200.0,0.15,400,0.936
3,sample_483,300,NaN,405.0,700,1:30:00,38.0,NaN,NaN,3:00:00,...,65,18:00:00,45.0,19:00-20:30,21:30-23:00,NaN,800.0,0.15,400,0.902
4,sample_617,300,NaN,405.0,700,22:00:00,29.0,NaN,NaN,0:00:00,...,80,9:00:00,45.0,9:00-10:30,10:30-12:00,12:00-13:00,1200.0,0.15,420,0.983


#### 变量的预处理

In [98]:
def standardize_value(value):
    """
    标准普通值，主要处理脏数据
    :param value: str
    """
    if isinstance(value, str) and value == '1900/3/10 0:00':
        value = 0
    return value


def preprocess_variable(df):
    """
    特征变量的预处理
    
    :param df: 未处理的特征变量
    :return: DataFrame
    """
    columns = ['A1', 'A2', 'A3', 'A4', 'A6', 'A8', 'A10', 'A12', 'A13', 'A15', 'A17', 'A18', 'A19', 
               'A21', 'A22', 'A23', 'A25', 'A27', 'B1', 'B2', 'B3', 'B6', 'B8', 'B12', 'B13', 'B14']
    result = df[columns].copy().fillna(0)
    for column in columns:
        result[column] = result[column].apply(lambda v: standardize_value(v))
    return result


df_standard_sample = preprocess_variable(df_sample)
print(df_standard_sample.shape)
df_standard_sample.head()

(1396, 26)


,A1,A2,A3,A4,A6,A8,A10,A12,A13,A15,...,A25,A27,B1,B2,B3,B6,B8,B12,B13,B14
0,300,0.0,405.0,700,38.0,0.0,100,102,0.2,103.0,...,75,70,350.0,3.5,3.5,65,45.0,800.0,0.15,400
1,300,0.0,405.0,700,29.0,0.0,101,103,0.2,104.0,...,80,73,320.0,3.5,3.5,80,45.0,1200.0,0.15,400
2,300,0.0,405.0,700,29.0,0.0,102,103,0.2,104.0,...,79,73,320.0,3.5,3.5,80,45.0,1200.0,0.15,400
3,300,0.0,405.0,700,38.0,0.0,100,102,0.2,103.0,...,70,78,290.0,3.5,3.5,65,45.0,800.0,0.15,400
4,300,0.0,405.0,700,29.0,0.0,101,103,0.2,104.0,...,80,73,320.0,3.5,3.5,80,45.0,1200.0,0.15,420


In [99]:
def standardize_time(value):
    """
    标准化时间的值，主要处理脏数据
    :param value: str
    """
    if isinstance(value, str):
        value = value.replace('"', ':').replace('\xa3\xbb', ':').replace('\xb7\xd6', '').replace(';', ':').replace('::', ':').replace('00:', '00').replace('1900/1/9 ', '').replace('1900/1/21 ', '').replace('1900/1/29 ', '').replace('1900/3/13 ', '').replace('1900/1/12 ', '').replace('1900/1/1 ', '').replace('1900/1/22 ', '').replace('1900/3/10 ', '')
    if isinstance(value, str) and re.match(r'^\d{1,2}00$', value):
        value = value.strip('00') + ':00'
    return value


def standardize_time_interval(value):
    """
    标准化时间区间的值，主要处理脏数据
    :param value: str
    """
    if isinstance(value, str) and re.match(r'^\d{2}:-\d{2}:\d{2}$', value):  # 19:-20:05
        value = value.split('-')[0] + '00-' + value.split('-')[1]
    if isinstance(value, str) and re.match(r'^:\d{2}:\d{2}$', value):  # :30:00
        value = '00' + value
    return value


def time_diff(start_time, end_time):
    """
    计算两个时间（HH:mm:ss 或 HH:mm）的差值（秒），任意一个为空的话，就返回0
    :param start_time: str，HH:mm:ss
    :param end_time: str，HH:mm:ss
    """
    if start_time is None or not isinstance(start_time, str):
        return None
    elif end_time is None or not isinstance(end_time, str):
        return None
    if start_time == '':
        return None
    if end_time == '':
        return None
    # 特殊情况，如：'15:00-1600'，'700'
    start_time = standardize_time(start_time)
    end_time = standardize_time(end_time)
    
    start_split = start_time.split(':')
    start_second = 3600*int(start_split[0]) + 60*(int(start_split[1])) + (int(start_split[2]) if len(start_split) >= 3 else 0)
    end_split = end_time.split(':')
    end_second = 3600*int(end_split[0]) + 60*(int(end_split[1])) + (int(end_split[2]) if len(end_split) >= 3 else 0)
    return end_second - start_second


def preprocess_time_variable(df):
    """
    时间类特征变量的预处理
    
    :param df: 未处理的特征变量
    :return: DataFrame
    """
    data, columns = [], []
    
    time_var_list = ['A5', 'A7', 'A9', 'A11', 'A14', 'A16', 'A20', 'A24', 'A26', 'A28', 
                     'B4', 'B5', 'B7', 'B9', 'B10', 'B11']
    time_interval_dict = {'A20': 1, 'A28': 1, 'B4': 1, 'B9': 1, 'B10': 1, 'B11': 1}
    
    # 1.统计 时间/时间段 到 时间/时间段 的情况
    had_append_columns = {}  # {column: 1}
    for i, s in df.iterrows():
        row = []
        for start_index in range(len(time_var_list)):
            start_column = time_var_list[start_index]
            end_index = start_index + 1
            
            over24_cnt = 0  # 转点的累计。开始转点时 +1
            values = []  # 记录所有的时间值，便于计算转点
            while end_index < len(time_var_list):
                end_column = time_var_list[end_index]
                
                start_value = s[start_column]
                end_value = s[end_column]
                # 脏数据处理
                start_value = standardize_time_interval(start_value)
                end_value = standardize_time_interval(end_value)
                
                if len(values) == 0 and isinstance(start_value, str):
                    values.append(start_value if '-' not in start_value else start_value.split('-'[0]))
                if start_column not in time_interval_dict and end_column not in time_interval_dict:  # 时间，时间
                    if isinstance(end_value, str):
                        values.append(end_value)
                        if len(values) >= 2 and time_diff(values[-2], values[-1]) < 0:  # 转点
                            over24_cnt += 1
                    
                    diff = time_diff(start_value, 
                                     end_value)
                    if diff is not None:
                        diff += over24_cnt * 24 * 3600
                    else:
                        diff = 0
                    column = '{}_{}'.format(start_column, end_column)
                    if column not in had_append_columns:
                        had_append_columns[column] = 1
                        columns.append(column)
                    row.append(diff)
                elif start_column not in time_interval_dict and end_column in time_interval_dict:  # 时间，时间段
                    # 时间，时间段-start
                    if isinstance(end_value, str):
                        values.append(end_value.split('-')[0])
                        if len(values) >= 2 and time_diff(values[-2], values[-1]) < 0:  # 转点
                            over24_cnt += 1
                    
                    diff = time_diff(start_value, 
                                     end_value.split('-')[0] if isinstance(end_value, str) else None)
                    if diff is not None:
                        diff += over24_cnt * 24 * 3600
                    else:
                        diff = 0
                    column = '{}_{}-start'.format(start_column, end_column)
                    if column not in had_append_columns:
                        had_append_columns[column] = 1
                        columns.append(column)
                    row.append(diff)
                    
                    # 时间，时间段-end
                    if isinstance(end_value, str):
                        values.append(end_value.split('-')[1])
                        if len(values) >= 2 and time_diff(values[-2], values[-1]) < 0:  # 转点
                            over24_cnt += 1
                    
                    diff = time_diff(start_value, 
                                     end_value.split('-')[1] if isinstance(end_value, str) else None)
                    if diff is not None:
                        diff += over24_cnt * 24 * 3600
                    else:
                        diff = 0
                    column = '{}_{}-end'.format(start_column, end_column)
                    if column not in had_append_columns:
                        had_append_columns[column] = 1
                        columns.append(column)
                    row.append(diff)
                elif start_column in time_interval_dict and end_column not in time_interval_dict:  # 时间段，时间
                    if isinstance(end_value, str):
                        values.append(end_value)
                        if len(values) >= 2 and time_diff(values[-2], values[-1]) < 0:  # 转点
                            over24_cnt += 1
                    
                    # 时间段-start，时间
                    diff = time_diff(start_value.split('-')[0] if isinstance(start_value, str) else None, 
                                     end_value)
                    if diff is not None:
                        diff += over24_cnt * 24 * 3600
                    else:
                        diff = 0
                    column = '{}-start_{}'.format(start_column, end_column)
                    if column not in had_append_columns:
                        had_append_columns[column] = 1
                        columns.append(column)
                    row.append(diff)
                    
                    # 时间段-end，时间
                    diff = time_diff(start_value.split('-')[1]  if isinstance(start_value, str) else None, 
                                     end_value)
                    if diff is not None:
                        diff += over24_cnt * 24 * 3600
                    else:
                        diff = 0
                    column = '{}-end{}'.format(start_column, end_column)
                    if column not in had_append_columns:
                        had_append_columns[column] = 1
                        columns.append(column)
                    row.append(diff)
                else:  # 时间段，时间段
                    # 时间段-start，时间段-start
                    if isinstance(end_value, str):
                        values.append(end_value.split('-')[0])
                        if len(values) >= 2 and time_diff(values[-2], values[-1]) < 0:  # 转点
                            over24_cnt += 1
                    diff = time_diff(start_value.split('-')[0] if isinstance(start_value, str) else None, 
                                     end_value.split('-')[0] if isinstance(end_value, str) else None)
                    if diff is not None:
                        diff += over24_cnt * 24 * 3600
                    else:
                        diff = 0
                    column = '{}-start_{}-start'.format(start_column, end_column)
                    if column not in had_append_columns:
                        had_append_columns[column] = 1
                        columns.append(column)
                    row.append(diff)
                    
                    # 时间段-end，时间段-start
                    diff = time_diff(start_value.split('-')[1] if isinstance(start_value, str) else None, 
                                     end_value.split('-')[0] if isinstance(end_value, str) else None)
                    if diff is not None:
                        diff += over24_cnt * 24 * 3600
                    else:
                        diff = 0
                    column = '{}-end{}-start'.format(start_column, end_column)
                    if column not in had_append_columns:
                        had_append_columns[column] = 1
                        columns.append(column)
                    row.append(diff)
                    
                    # 时间段-start，时间段-end
                    if isinstance(end_value, str):
                        values.append(end_value.split('-')[1])
                        if len(values) >= 2 and time_diff(values[-2], values[-1]) < 0:  # 转点
                            over24_cnt += 1
                    diff = time_diff(start_value.split('-')[0] if isinstance(start_value, str) else None, 
                                     end_value.split('-')[1] if isinstance(end_value, str) else None)
                    if diff is not None:
                        diff += over24_cnt * 24 * 3600
                    else:
                        diff = 0
                    column = '{}-start_{}-end'.format(start_column, end_column)
                    if column not in columns:
                        columns.append(column)
                    row.append(diff)
                    
                    # 时间段-end，时间段-end
                    diff = time_diff(start_value.split('-')[1] if isinstance(start_value, str) else None, 
                                     end_value.split('-')[1] if isinstance(end_value, str) else None)
                    if diff is not None:
                        diff += over24_cnt * 24 * 3600
                    else:
                        diff = 0
                    column = '{}-end_{}-end'.format(start_column, end_column)
                    if column not in columns:
                        columns.append(column)
                    row.append(diff)
                
                end_index += 1
            pass
        data.append(row)
    
    # 2.统计 时间段 的值
    result = pd.DataFrame(data, columns=columns)
    for column in sorted(time_interval_dict.keys()):
        time_interval_values = df_sample[column].apply(lambda time_interval: time_diff(standardize_time_interval(time_interval).split('-')[0], standardize_time_interval(time_interval).split('-')[1]) if isinstance(time_interval, str) else 0)
        result[column] = time_interval_values
    
    return result


df_time_sample = preprocess_time_variable(df_sample)
print(df_time_sample.shape)
df_time_sample.head()

(1396, 231)


,A5_A7,A5_A9,A5_A11,A5_A14,A5_A16,A5_A20-start,A5_A20-end,A5_A24,A5_A26,A5_A28-start,...,B10-start_B11-start,B10-endB11-start,B10-start_B11-end,B10-end_B11-end,A20,A28,B10,B11,B4,B9
0,0,7200,10800,14400,18000,27000,28800,30600,32400,61200,...,0,0,0,0,1800,1800,5400,0,3600,5400
1,0,7200,10800,14400,18000,18000,21600,21600,25200,25200,...,91800,86400,95400,90000,3600,3600,5400,3600,3600,5400
2,0,7200,10800,14400,18000,18000,19800,21600,25200,25200,...,91800,86400,95400,90000,1800,3600,5400,3600,3600,5400
3,0,5400,9000,12600,16200,18000,19800,21600,23400,43200,...,0,0,0,0,1800,3600,5400,0,3600,5400
4,0,7200,10800,14400,18000,18000,21600,21600,25200,25200,...,91800,86400,95400,90000,3600,3600,5400,3600,3600,5400


#### 导出预处理后的样本

###### 训练集

In [100]:
train_pre_file_path = '/home/centos/leon/machine_learning_jupyter/dataset/jinnan_round1_train_20181227_preprocess_20190108.csv'

df_train = df_standard_sample.copy()
df_train.insert(0, 'id', df_sample['id'].values.tolist())
for column in df_time_sample.columns.tolist():
    df_train[column] = df_time_sample[column]
df_train['y'] = df_sample['y']

In [101]:
df_train.to_csv(train_pre_file_path, index=False)

###### 测试集

In [102]:
test_file_path = '/home/centos/leon/machine_learning_jupyter/dataset/jinnan_round1_testA_20181227.csv'
test_pre_file_path = '/home/centos/leon/machine_learning_jupyter/dataset/jinnan_round1_testA_20181227_preprocess_20190108.csv'

df_test_sample = pd.read_csv(test_file_path)
df_test_sample = pd.DataFrame(df_test_sample.values.tolist(), columns=(['id'] + df_test_sample.iloc[:, 1:].columns.tolist()))
df_standard_test_sample = preprocess_variable(df_test_sample)
df_time_test_sample = preprocess_time_variable(df_test_sample)

df_test = df_standard_test_sample.copy()
df_test.insert(0, 'id', df_test_sample['id'].values.tolist())
for column in df_time_test_sample.columns.tolist():
    df_test[column] = df_time_test_sample[column]


In [103]:
df_test.to_csv(test_pre_file_path, index=False)